In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
data = pd.read_csv("/Users/chitobadillo/Desktop/sp19-222-92/project/gamedata/total_game_data2.csv")
data.head()

,HR min [bpm],HR avg [bpm],HR max [bpm],HR min [%],HR avg [%],HR max [%],Time in HR zone 1 (50 - 59 %),Time in HR zone 2 (60 - 69 %),Time in HR zone 3 (70 - 79 %),Time in HR zone 4 (80 - 89 %),...,Number of accelerations (-1.99 - -1.00 m/s),Number of accelerations (-0.99 - -0.50 m/s),Number of accelerations (0.50 - 0.99 m/s),Number of accelerations (1.00 - 1.99 m/s),Number of accelerations (2.00 - 2.99 m/s),Number of accelerations (3.00 - 50.00 m/s),Calories [kcal],Training load score,Recovery time [h],Min Played
0,37,148,210,19,74,105,0:19:22,0:27:11,0:56:50,0:52:40,...,399,762,593,389,119,19,1343,230,46.0,99
1,62,98,188,31,49,94,0:23:49,0:08:11,0:08:23,0:02:33,...,82,239,211,69,12,0,495,30,5.5,0
2,80,150,217,40,75,109,0:28:32,0:20:40,0:15:03,0:35:51,...,302,601,500,333,97,11,1407,289,135.7,82
3,86,150,200,43,75,100,0:19:42,0:15:29,0:19:19,0:35:16,...,352,590,647,360,77,12,1462,302,151.4,79
4,83,121,192,42,61,96,1:23:38,0:25:26,0:11:26,0:10:58,...,129,325,276,118,21,7,1019,118,21.3,7


In [3]:
# Removes every row where a player had zero minutes played
data = data[data['Min Played'] != 0]

In [4]:
data = data[data.columns.drop(list(data.filter(regex='Time')))]
data = data[data.columns.drop(list(data.filter(regex='HR')))]
data = data[data.columns.drop(list(data.filter(regex='Calories')))]
data = data[data.columns.drop(list(data.filter(regex='load score')))]
data = data[data.columns.drop(list(data.filter(regex='Recovery')))]
data = data[data.columns.drop(list(data.filter(regex='Speed zone 1')))]

data.head()

,Total distance [yd],Distance / min [yd/min],Maximum speed [mph],Average speed [mph],Sprints,Distance in Speed zone 2 [yd] (0.10 - 2.59 mph),Distance in Speed zone 3 [yd] (2.60 - 5.13 mph),Distance in Speed zone 4 [yd] (5.14 - 8.38 mph),Distance in Speed zone 5 [yd] (8.39- mph),Number of accelerations (-50.00 - -3.00 m/s),Number of accelerations (-2.99 - -2.00 m/s),Number of accelerations (-1.99 - -1.00 m/s),Number of accelerations (-0.99 - -0.50 m/s),Number of accelerations (0.50 - 0.99 m/s),Number of accelerations (1.00 - 1.99 m/s),Number of accelerations (2.00 - 2.99 m/s),Number of accelerations (3.00 - 50.00 m/s),Min Played
0,15935,98,18.7,3.5,37,1595,4651,5321,4366,27,72,399,762,593,389,119,19,99
2,11568,71,18.3,2.6,20,1464,3483,3802,2815,27,109,302,601,500,333,97,11,82
3,11981,74,16.4,2.7,23,1503,3750,3758,2969,25,94,352,590,647,360,77,12,79
4,3133,19,16.3,0.7,9,683,783,1034,628,7,25,129,325,276,118,21,7,7
5,13265,81,17.9,2.9,19,1501,5030,4573,2158,28,84,332,671,629,379,78,14,92


In [5]:
# Column names to use in later loop
distance_columns = ('Distance in Speed zone 2 [yd] (0.10 - 2.59 mph)', 'Distance in Speed zone 3 [yd] (2.60 - 5.13 mph)', 'Distance in Speed zone 4 [yd] (5.14 - 8.38 mph)', 'Distance in Speed zone 5 [yd] (8.39- mph)')
accel_columns = ('Number of accelerations (-50.00 - -3.00 m/s)', 'Number of accelerations (-2.99 - -2.00 m/s)', 'Number of accelerations (-1.99 - -1.00 m/s)', 'Number of accelerations (-0.99 - -0.50 m/s)', 'Number of accelerations (0.50 - 0.99 m/s)', 'Number of accelerations (1.00 - 1.99 m/s)', 'Number of accelerations (2.00 - 2.99 m/s)', 'Number of accelerations (3.00 - 50.00 m/s)')

# Create new column for total accelerations
data['Total accelerations'] = pd.Series(np.random.randn(len(data)), index=data.index)

# Convert column dtypes to floats for percentage values
data['Sprints'] = data['Sprints'].apply(np.float64)
for i in distance_columns:
    data[i] = data[i].apply(np.float64)
for i in accel_columns:
    data[i] = data[i].apply(np.float64)

# Adjust sprints for minutes played, adjust speed zones for total distance, and get total accelerations
for index, row in data.iterrows():
    # Initialize total acceleration value
    total_accels = 0
    # Change # of sprints to sprints per min
    data.at[index, 'Sprints'] = row['Sprints'] / row['Min Played']
    # Divide distance in speed zone by total distance, save back in speed zone column
    for i in distance_columns:
        data.at[index, i] = (row[i]) / int(row[0])
    # Calculate total # of accelerations and save in new column
    for i in accel_columns:
        total_accels += row[i]
    data.at[index, 'Total accelerations'] = total_accels

# Second loop to divide each acceleration column by the total accelerations, save back in acceleration columns
for index, row in data.iterrows():
    for i in accel_columns:
        data.at[index, i] = (row[i]) / int(row['Total accelerations'])
    
data.head()

# Save changes in new csv to view later. Will be moved to after normalization
data.to_csv("/Users/chitobadillo/Desktop/sp19-222-92/project/gamedata/altered_total.csv")

In [6]:
# Normalize
from sklearn import preprocessing

x = data.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data = pd.DataFrame(x_scaled)

In [7]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.968666,1.000000,0.177370,0.93750,0.146825,0.025946,0.398065,0.533631,1.000000,0.543941,0.365080,0.669382,0.384289,0.124007,0.616731,0.836389,0.570906,0.923810,0.628571
1,0.651918,0.682353,0.165138,0.65625,0.095819,0.131474,0.457917,0.510993,0.829093,0.664080,0.923193,0.472212,0.262531,0.157957,0.668678,0.812174,0.397297,0.761905,0.465306
2,0.681874,0.717647,0.107034,0.68750,0.114376,0.127058,0.535242,0.446352,0.853996,0.556817,0.665016,0.611262,0.042866,0.636386,0.654603,0.493521,0.397850,0.733333,0.537551
3,0.040110,0.070588,0.103976,0.06250,0.505102,0.496154,0.125605,0.516896,0.589852,0.353377,0.303885,0.335894,0.660718,0.676848,0.249438,0.192182,0.551316,0.047619,0.027755
4,0.775005,0.800000,0.152905,0.75000,0.081134,0.078032,0.965157,0.580265,0.379243,0.611907,0.537727,0.437814,0.258133,0.475182,0.700750,0.481936,0.452004,0.857143,0.561224


In [42]:
#Shuffling the data by column

random.shuffle(data[0])
random.shuffle(data[1])
random.shuffle(data[2])
random.shuffle(data[3])
random.shuffle(data[4])
random.shuffle(data[5])
random.shuffle(data[6])
random.shuffle(data[7])
random.shuffle(data[8])
random.shuffle(data[9])
random.shuffle(data[10])
random.shuffle(data[11])
random.shuffle(data[12])
random.shuffle(data[13])
random.shuffle(data[14])
random.shuffle(data[15])
random.shuffle(data[16])
random.shuffle(data[17])
random.shuffle(data[18])

In [44]:
x_train = data[0:135]
x_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.247117,0.458824,0.177370,0.50000,0.088054,0.981290,0.602157,0.530973,0.459153,0.664080,0.287066,0.319109,0.150453,0.660871,0.794267,0.585651,0.067721,0.733333,0.677551
1,0.281642,0.717647,0.186544,0.15625,0.055138,0.329541,0.636725,0.499930,0.820031,0.428137,0.213780,0.671012,0.153359,0.405711,0.445542,0.561752,0.279350,0.104762,0.495918
2,0.121201,0.129412,0.097859,0.78125,0.056746,0.123837,0.487628,0.724707,0.599267,0.469932,0.550175,0.614024,0.387316,0.495748,0.375427,0.101884,0.514897,0.847619,0.628163
3,0.607311,0.070588,0.045872,0.68750,0.183333,0.039143,0.596591,0.577680,0.664136,0.314779,0.571816,0.530760,0.574003,0.445907,0.178763,0.364942,0.454260,0.123810,0.228980
4,0.692029,0.400000,0.085627,0.46875,0.055073,0.425503,0.457917,0.427267,0.778881,0.370570,0.293019,0.327024,0.275323,0.692348,0.296052,0.192182,0.239290,0.600000,0.362041
5,0.691231,0.482353,0.088685,0.40625,0.150080,0.867234,0.553454,0.510993,0.344472,0.300677,0.483007,0.468715,0.380230,0.358696,0.467622,0.534222,0.214112,0.790476,0.622857
6,0.374411,0.541176,0.168196,0.75000,0.238520,0.210213,0.330116,0.693283,0.842837,0.222278,0.225552,0.693629,0.251122,0.418238,0.290992,0.414117,0.000000,0.761905,0.557551
7,0.767027,0.705882,0.180428,0.15625,0.042243,0.510322,0.525971,0.303331,0.251940,0.383863,0.446625,0.867387,0.253182,0.569057,0.437561,0.512467,0.796364,0.000000,0.155918
8,0.551099,0.717647,0.110092,0.75000,0.148649,0.178381,0.900263,0.418493,0.209101,0.328960,0.435030,0.653081,0.262531,0.062924,0.422240,0.113253,0.716438,0.895238,0.607755
9,0.425256,0.635294,0.073394,0.12500,0.163690,0.920536,0.711214,0.541730,0.374561,0.330276,0.196059,0.503839,0.396346,0.368871,0.596987,0.718155,0.293329,0.552381,0.640000
